In [13]:
# !pip install pandas pyodbc


In [14]:
import pandas as pd
import pyodbc

# Define the connection string to your SQL Server
conn_str = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=MY-PC\\SQLEXPRESS;"  # Use the server name from the image
    "DATABASE=DB-Test;"                 # Replace with your actual database name
    "Trusted_Connection=yes;"            # This enables Windows Authentication
)

# Establish the connection
conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

# Load the CSV file into a pandas DataFrame
csv_file_path = 'test2.csv'
df = pd.read_csv(csv_file_path)

# Define the name of the table where data will be inserted
table_name = 'test_table'

# Check if the table exists, and create it if it does not
cursor.execute(f"""
IF NOT EXISTS (SELECT * FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = '{table_name}')
BEGIN
    CREATE TABLE {table_name} (
        {', '.join([f'{col} NVARCHAR(MAX)' for col in df.columns])}  -- Define columns based on CSV headers
    )
END
""")
conn.commit()

# Insert DataFrame to SQL Server
for index, row in df.iterrows():
    # Assuming your table has columns that match your CSV file columns
    sql = f"INSERT INTO {table_name} VALUES ({','.join(['?' for _ in row])})"
    cursor.execute(sql, tuple(row))

# Commit the transaction
conn.commit()

# Close the connection
cursor.close()
conn.close()

print("Data has been inserted successfully.")


Data has been inserted successfully.
